# Database migration rules

In [1]:
from fludashboard import settings

import fludashboard as flud
import glob
import os
import pandas as pd
import sqlalchemy as sqla

In [2]:
# pandas configuration
pd.set_option('display.max_columns', 99)

In [3]:
def get_filename_from_path(file_path: str):
    """
    """
    return file_path.split(os.path.sep)[-1].split('.')[0]

In [4]:
path_data_files = os.path.join(flud.__path__[0], 'data', '*.csv')

In [5]:
dfs = {}
pks = {}

In [6]:
print('Data files:')
for file_path in glob.glob(path_data_files):
    filename = get_filename_from_path(file_path)
    
    print(filename)
    
    dfs[filename] = pd.read_csv(file_path)

Data files:
mem-report
current_estimated_values
clean_data_epiweek-weekly-incidence_w_situation


/home/xmn/miniconda3/envs/fludashboard/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


historical_estimated_values
mem-typical


## 1. Setting IDs

### 1.1 Datasets

In [7]:
dataset_id = {
    'srag': 1,
    'sragflu': 2,
    'obitoflu': 3,
}

### 1.2 Scale

In [8]:
scale_id = {
    'incidência': 1,
    'casos': 2
}

### 1.3 Situation

In [9]:
situation_id = {
    'unknown': 1, 
    'estimated': 2, 
    'stable': 3
}

### 1.4 Territory Type

In [10]:
territory_type_id = {
    'Estado': 1, 
    'Regional': 2,
    'Região': 3,
    'País': 4
}

### 1.5 Region id conversion

In [11]:
region_id = {
    'BR': 0,
    'RegN': 1001,
    'RegL': 1002,
    'RegC': 1003,
    'RegS': 1004
}

### Territory Table

In [12]:
df_territory = pd.DataFrame([
    {'id': '11', 'initials': 'RO', 'name': 'Rondônia', 'territory_type': 1},
    {'id': '12', 'initials': 'AC', 'name': 'Acre', 'territory_type': 1},
    {'id': '13', 'initials': 'AM', 'name': 'Amazonas', 'territory_type': 1},
    {'id': '14', 'initials': 'RR', 'name': 'Roraima', 'territory_type': 1},
    {'id': '15', 'initials': 'RO', 'name': 'Pará', 'territory_type': 1},
    {'id': '16', 'initials': 'RO', 'name': 'Amapá', 'territory_type': 1},
    {'id': '17', 'initials': 'RO', 'name': 'Tocantins', 'territory_type': 1},
    {'id': '21', 'initials': 'RO', 'name': 'Maranhão', 'territory_type': 1},
    {'id': '22', 'initials': 'RO', 'name': 'Piauí', 'territory_type': 1},
    {'id': '23', 'initials': 'RO', 'name': 'Ceará', 'territory_type': 1},
    {'id': '24', 'initials': 'RO', 'name': 'Rio Grande do Norte', 'territory_type': 1},
    {'id': '25', 'initials': 'RO', 'name': 'Paraíba', 'territory_type': 1},
    {'id': '26', 'initials': 'RO', 'name': 'Pernambuco', 'territory_type': 1},
    {'id': '27', 'initials': 'RO', 'name': 'Alagoas', 'territory_type': 1},
    {'id': '28', 'initials': 'RO', 'name': 'Sergipe', 'territory_type': 1},
    {'id': '29', 'initials': 'RO', 'name': 'Bahia', 'territory_type': 1},
    {'id': '31', 'initials': 'RO', 'name': 'Minas Gerais', 'territory_type': 1},
    {'id': '32', 'initials': 'RO', 'name': 'Espírito Santo', 'territory_type': 1},
    {'id': '33', 'initials': 'RO', 'name': 'Rio de Janeiro', 'territory_type': 1},
    {'id': '35', 'initials': 'RO', 'name': 'São Paulo', 'territory_type': 1},
    {'id': '41', 'initials': 'RO', 'name': 'Paraná', 'territory_type': 1},
    {'id': '42', 'initials': 'RO', 'name': 'Santa Catarina', 'territory_type': 1},
    {'id': '43', 'initials': 'RO', 'name': 'Rio Grande do Sul', 'territory_type': 1},
    {'id': '50', 'initials': 'RO', 'name': 'Mato Grosso do Sul', 'territory_type': 1},
    {'id': '51', 'initials': 'RO', 'name': 'Mato Grosso', 'territory_type': 1},
    {'id': '52', 'initials': 'RO', 'name': 'Goiás', 'territory_type': 1},
    {'id': '53', 'initials': 'RO', 'name': 'Distrito Federal', 'territory_type': 1},
    {'id': '0', 'initials': 'BR', 'name': 'Brasil', 'territory_type': 4},
    {'id': '1003', 'initials': 'RegC', 'name': 'Regional Centro', 'territory_type': 2},
    {'id': '1002', 'initials': 'RegL', 'name': 'Regional Leste', 'territory_type': 2},
    {'id': '1001', 'initials': 'RegN', 'name': 'Regional Norte', 'territory_type': 2},
    {'id': '1004', 'initials': 'RegS', 'name': 'Regional Sul', 'territory_type': 2},
    {'id': '1', 'initials': 'N', 'name': 'Norte', 'territory_type': 3},
    {'id': '2', 'initials': 'NE', 'name': 'Nordeste', 'territory_type': 3},
    {'id': '3', 'initials': 'SE', 'name': 'Sudeste', 'territory_type': 3},
    {'id': '5', 'initials': 'CO', 'name': 'Centro-oeste', 'territory_type': 3},
    {'id': '4', 'initials': 'S', 'name': 'Sul', 'territory_type': 3},
])

df_territory.set_index('id', inplace=True)

df_territory

,initials,name,territory_type
id,,,
11,RO,Rondônia,1
12,AC,Acre,1
13,AM,Amazonas,1
14,RR,Roraima,1
15,RO,Pará,1
16,RO,Amapá,1
17,RO,Tocantins,1
21,RO,Maranhão,1
22,RO,Piauí,1


## 2. current_estimated_values

In [13]:
dataset = 'current_estimated_values'

In [14]:
dfs[dataset].head()

,UF,epiyear,epiweek,SRAG,Tipo,Situation,mean,50%,2.5%,97.5%,L0,L1,L2,L3,Run date,dado,escala
0,11,2009,1,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
1,11,2009,2,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
2,11,2009,3,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
3,11,2009,4,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência
4,11,2009,5,0.0,Estado,stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,srag,incidência


In [15]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'territory_type',  # Not needed in the table
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',
    'escala': 'scale'
}

# rename columns
dfs[dataset].rename(
    columns=migration_rules, inplace=True
)

# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)
regions_indeces = dfs[dataset].state_country.isin([
    'BR', 'RegN', 'RegL', 'RegC', 'RegS'
])
dfs[dataset].loc[regions_indeces, 'state_country'] = dfs[dataset].loc[
    regions_indeces, 'state_country'
].map(region_id)
dfs[dataset].state_country = dfs[dataset].state_country.astype(int)

# remove unnecessary fields
dfs[dataset].drop(['territory_type'], axis=1, inplace=True)

dfs[dataset].head()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,dataset,scale
0,11,2009,1,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
1,11,2009,2,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
2,11,2009,3,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
3,11,2009,4,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1
4,11,2009,5,0.0,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2017-10-11,1,1


In [16]:
dfs[dataset].tail()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,dataset,scale
86203,1004,2017,27,9.0,2,10.0,9.0,9.0,11.0,0.000,0.988,0.012,0.000,2017-10-11,3,2
86204,1004,2017,28,8.0,2,8.0,8.0,7.0,10.0,0.800,0.200,0.000,0.000,2017-10-11,3,2
86205,1004,2017,29,9.0,2,10.0,10.0,9.0,13.0,0.000,0.856,0.140,0.004,2017-10-11,3,2
86206,1004,2017,30,2.0,2,2.0,2.0,0.0,7.0,0.996,0.004,0.000,0.000,2017-10-11,3,2
86207,1004,2017,31,3.0,2,9.0,8.0,3.0,18.0,0.560,0.256,0.136,0.048,2017-10-11,3,2


In [17]:
dfs[dataset].dtypes

state_country      int64
epiyear            int64
epiweek            int64
value            float64
situation          int64
mean             float64
median           float64
ci_lower         float64
ci_upper         float64
low              float64
epidemic         float64
high             float64
very_high        float64
run_date          object
dataset            int64
scale              int64
dtype: object

In [18]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value  situation  mean  median  \
dataset scale state_country epiyear epiweek                                   
1       1     11            2009    1          0.0          3   0.0     0.0   
                                    2          0.0          3   0.0     0.0   
                                    3          0.0          3   0.0     0.0   
                                    4          0.0          3   0.0     0.0   
                                    5          0.0          3   0.0     0.0   

                                             ci_lower  ci_upper  low  \
dataset scale state_country epiyear epiweek                            
1       1     11            2009    1             0.0       0.0  1.0   
                                    2             0.0       0.0  1.0   
                                    3             0.0       0.0  1.0   
                                    4             0.0       0.0  1.0   
                                    5             0.0       0.0  1.0   

                                             epidemic  high  very_high  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1             0.0   0.0        0.0   
                                    2             0.0   0.0        0.0   
                                    3             0.0   0.0        0.0   
                                    4             0.0   0.0        0.0   
                                    5             0.0   0.0        0.0   

                                               run_date  
dataset scale state_country epiyear epiweek              
1       1     11            2009    1        2017-10-11  
                                    2        2017-10-11  
                                    3        2017-10-11  
                                    4        2017-10-11  
                                    5        2017-10-11

## 3. historical_estimated_values

In [19]:
dataset = 'historical_estimated_values'

In [20]:
dfs[dataset].head()

,UF,epiyear,epiweek,SRAG,Tipo,Situation,mean,50%,2.5%,97.5%,L0,L1,L2,L3,Run date,base_epiyearweek,base_epiyear,base_epiweek,dado,escala
0,11,2017,7,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.055377,0.976,0.024,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
1,11,2017,8,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.110755,0.972,0.028,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
2,11,2017,9,0.000000,Estado,estimated,0.000000,0.000000,0.000000,0.000000,1.000,0.000,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
3,11,2017,10,0.000000,Estado,estimated,0.000000,0.000000,0.000000,0.055377,0.996,0.004,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência
4,11,2017,11,0.055377,Estado,estimated,0.055377,0.055377,0.055377,0.110755,0.964,0.036,0.0,0.0,2017-10-11,2017W23,2017,23,srag,incidência


In [21]:
migration_rules = {
    'UF': 'state_country',
    'SRAG': 'value',
    'Tipo': 'territory_type',  # Not needed in the table
    'Situation': 'situation',
    '50%': 'median',
    '2.5%': 'ci_lower',
    '97.5%': 'ci_upper',
    'L0': 'low',
    'L1': 'epidemic',
    'L2': 'high',
    'L3': 'very_high',
    'Run date': 'run_date',
    'dado': 'dataset',  # or origin
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)
regions_indeces = dfs[dataset].state_country.isin([
    'BR', 'RegN', 'RegL', 'RegC', 'RegS'
])
dfs[dataset].loc[regions_indeces, 'state_country'] = dfs[dataset].loc[
    regions_indeces, 'state_country'
].map(region_id)
dfs[dataset].state_country = dfs[dataset].state_country.astype(int)

# remove unnecessary fields
dfs[dataset].drop(['territory_type'], axis=1, inplace=True)

dfs[dataset].head()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,base_epiyearweek,base_epiyear,base_epiweek,dataset,scale
0,11,2017,7,0.055377,2,0.055377,0.055377,0.055377,0.055377,0.976,0.024,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
1,11,2017,8,0.055377,2,0.055377,0.055377,0.055377,0.110755,0.972,0.028,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
2,11,2017,9,0.000000,2,0.000000,0.000000,0.000000,0.000000,1.000,0.000,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
3,11,2017,10,0.000000,2,0.000000,0.000000,0.000000,0.055377,0.996,0.004,0.0,0.0,2017-10-11,2017W23,2017,23,1,1
4,11,2017,11,0.055377,2,0.055377,0.055377,0.055377,0.110755,0.964,0.036,0.0,0.0,2017-10-11,2017W23,2017,23,1,1


In [22]:
dfs[dataset].tail()

,state_country,epiyear,epiweek,value,situation,mean,median,ci_lower,ci_upper,low,epidemic,high,very_high,run_date,base_epiyearweek,base_epiyear,base_epiweek,dataset,scale
20567,1004,2017,27,9.0,2,10.0,9.0,9.0,11.0,0.000,0.988,0.012,0.000,2017-10-11,2017W31,2017,31,3,2
20568,1004,2017,28,8.0,2,8.0,8.0,7.0,10.0,0.800,0.200,0.000,0.000,2017-10-11,2017W31,2017,31,3,2
20569,1004,2017,29,9.0,2,10.0,10.0,9.0,13.0,0.000,0.856,0.140,0.004,2017-10-11,2017W31,2017,31,3,2
20570,1004,2017,30,2.0,2,2.0,2.0,0.0,7.0,0.996,0.004,0.000,0.000,2017-10-11,2017W31,2017,31,3,2
20571,1004,2017,31,3.0,2,9.0,8.0,3.0,18.0,0.560,0.256,0.136,0.048,2017-10-11,2017W31,2017,31,3,2


In [23]:
dfs[dataset].dtypes

state_country         int64
epiyear               int64
epiweek               int64
value               float64
situation             int64
mean                float64
median              float64
ci_lower            float64
ci_upper            float64
low                 float64
epidemic            float64
high                float64
very_high           float64
run_date             object
base_epiyearweek     object
base_epiyear          int64
base_epiweek          int64
dataset               int64
scale                 int64
dtype: object

In [24]:
# primary_keys
pks[dataset] = [
    'dataset', 'scale', 'state_country', 
    'base_epiyear', 'base_epiweek', 
    'epiyear', 'epiweek'
]

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

value  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       situation  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek              
1       1     11            2017         23           2017    7                2   
                                                              8                2   
                                                              9                2   
                                                              10               2   
                                                              11               2   

                                                                           mean  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                         median  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_lower  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.055377   
                                                              9        0.000000   
                                                              10       0.000000   
                                                              11       0.055377   

                                                                       ci_upper  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek             
1       1     11            2017         23           2017    7        0.055377   
                                                              8        0.110755   
                                                              9        0.000000   
                                                              10       0.055377   
                                                              11       0.110755   

                                                                         low  \
dataset scale state_country base_epiyear base_epiweek epiyear epiweek          
1       1     11            2017         23           2017    7        0.976   
                                                              8        0.972   
                                                              9        1.000   
                                                              10       0.996   
                                                              11       0.964   

              

## 4. clean_data_epiweek-weekly-incidence_w_situation

In [25]:
dataset = 'clean_data_epiweek-weekly-incidence_w_situation'

In [26]:
dfs[dataset].head()

,0-4 anos,10-19 anos,2-4 anos,20-29 anos,30-39 anos,40-49 anos,5-9 anos,50-59 anos,60+ anos,< 2 anos,DELAYED,FLU_A,FLU_B,INCONCLUSIVE,Idade desconhecida,NEGATIVE,NOTTESTED,OTHERS,POSITIVE_CASES,SRAG,Situation,TESTING_IGNORED,Tipo,UF,Unidade da Federação,VSR,dado,epiweek,epiyear,epiyearweek,escala,sexo
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,F
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,M
2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,1,2009,2009W01,incidência,Total
3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,2,2009,2009W02,incidência,F
4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,stable,0.0,Estado,11,Rondônia,0.0,srag,2,2009,2009W02,incidência,M


In [27]:
dfs[dataset].keys()

Index(['0-4 anos', '10-19 anos', '2-4 anos', '20-29 anos', '30-39 anos',
       '40-49 anos', '5-9 anos', '50-59 anos', '60+ anos', '< 2 anos',
       'DELAYED', 'FLU_A', 'FLU_B', 'INCONCLUSIVE', 'Idade desconhecida',
       'NEGATIVE', 'NOTTESTED', 'OTHERS', 'POSITIVE_CASES', 'SRAG',
       'Situation', 'TESTING_IGNORED', 'Tipo', 'UF', 'Unidade da Federação',
       'VSR', 'dado', 'epiweek', 'epiyear', 'epiyearweek', 'escala', 'sexo'],
      dtype='object')

In [28]:
migration_rules = {
    '0-4 anos': 'years_0_4',
    '10-19 anos': 'years_10_19',
    '2-4 anos': 'years_2_4',
    '20-29 anos': 'years_20_29',
    '30-39 anos': 'years_30_39',
    '40-49 anos': 'years_40_49',
    '5-9 anos': 'years_5_9',
    '50-59 anos': 'years_50_59',
    '60+ anos': 'years_60_or_more',
    '< 2 anos': 'years_lt_2',
    'DELAYED': 'delayed',
    'FLU_A': 'flu_a',
    'FLU_B': 'flu_b',
    'INCONCLUSIVE': 'inconclusive',
    'Idade desconhecida': 'unknown_age',
    'NEGATIVE': 'negative',
    'NOTTESTED': 'not_tested',
    'OTHERS': 'others',
    'POSITIVE_CASES': 'positive_cases',
    'SRAG': 'value',
    'Situation': 'situation',
    'TESTING_IGNORED': 'testing_ignored',
    'Tipo': 'territory_type',  # Not needed in the table
    'UF': 'state_country',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'VSR': 'vsr',
    'dado': 'dataset',
    'escala': 'scale',
    'sexo': 'gender'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
dfs[dataset].situation = dfs[dataset].situation.map(situation_id)
regions_indeces = dfs[dataset].state_country.isin([
    'BR', 'RegN', 'RegL', 'RegC', 'RegS'
])
dfs[dataset].loc[regions_indeces, 'state_country'] = dfs[dataset].loc[
    regions_indeces, 'state_country'
].map(region_id)
dfs[dataset].state_country = dfs[dataset].state_country.astype(int)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)


dfs[dataset].head()

,years_0_4,years_10_19,years_2_4,years_20_29,years_30_39,years_40_49,years_5_9,years_50_59,years_60_or_more,years_lt_2,delayed,flu_a,flu_b,inconclusive,unknown_age,negative,not_tested,others,positive_cases,value,situation,testing_ignored,state_country,vsr,dataset,epiweek,epiyear,epiyearweek,scale,gender
0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,F
1,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,M
2,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,1,2009,2009W01,1,Total
3,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,2,2009,2009W02,1,F
4,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,3,0.0,11,0.0,1,2,2009,2009W02,1,M


In [29]:
dfs[dataset].tail()

,years_0_4,years_10_19,years_2_4,years_20_29,years_30_39,years_40_49,years_5_9,years_50_59,years_60_or_more,years_lt_2,delayed,flu_a,flu_b,inconclusive,unknown_age,negative,not_tested,others,positive_cases,value,situation,testing_ignored,state_country,vsr,dataset,epiweek,epiyear,epiyearweek,scale,gender
301723,NaN,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1,0.0,1004,0.0,3,30,2017,2017W30,2,Total
301724,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.0,1004,0.0,3,31,2017,2017W31,2,F
301725,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,1004,0.0,3,31,2017,2017W31,2,I
301726,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1,0.0,1004,0.0,3,31,2017,2017W31,2,M
301727,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,1,0.0,1004,0.0,3,31,2017,2017W31,2,Total


In [30]:
dfs[dataset].dtypes

years_0_4           float64
years_10_19         float64
years_2_4           float64
years_20_29         float64
years_30_39         float64
years_40_49         float64
years_5_9           float64
years_50_59         float64
years_60_or_more    float64
years_lt_2          float64
delayed             float64
flu_a               float64
flu_b               float64
inconclusive        float64
unknown_age         float64
negative            float64
not_tested          float64
others              float64
positive_cases      float64
value               float64
situation             int64
testing_ignored     float64
state_country         int64
vsr                 float64
dataset               int64
epiweek               int64
epiyear               int64
epiyearweek          object
scale                 int64
gender               object
dtype: object

In [31]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'epiyear', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

years_0_4  years_10_19  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             years_2_4  years_20_29  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              NaN          0.0   
                                    1              NaN          0.0   
                                    1              NaN          0.0   
                                    2              NaN          0.0   
                                    2              NaN          0.0   

                                             years_30_39  years_40_49  \
dataset scale state_country epiyear epiweek                             
1       1     11            2009    1                0.0          0.0   
                                    1                0.0          0.0   
                                    1                0.0          0.0   
                                    2                0.0          0.0   
                                    2                0.0          0.0   

                                             years_5_9  years_50_59  \
dataset scale state_country epiyear epiweek                           
1       1     11            2009    1              0.0          0.0   
                                    1              0.0          0.0   
                                    1              0.0          0.0   
                                    2              0.0          0.0   
                                    2              0.0          0.0   

                                             years_60_or_more  years_lt_2  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    1                     0.0         NaN   
                                    2                     0.0         NaN   
                                    2                     0.0         NaN   

                                             delayed  flu_a  flu_b  \
dataset scale state_country epiyear epiweek                          
1       1     11            2009    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    1            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   
                                    2            0.0    0.0    0.0   

                                             inconclusive  unknown_age  \
dataset scale state_country epiyear epiweek                              
1       1     11            2009    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    1                 0.0          NaN   
                                    2                 0.0          NaN   
                                    2                 0.0          NaN   

                                             negative  not_tested  others  \
dataset scale state_country epiyear epiweek                                 
1       1     11            2009    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    1             0.0         0.0     0.0   
                                    2             0.0         0.0     0.0   
                           

## 5. mem-report

In [32]:
dataset = 'mem-report'

In [33]:
dfs[dataset].head()

,UF,População,Média geométrica do pico de infecção das temporadas regulares,região de baixa atividade típica,limiar pré-epidêmico,intensidade alta,intensidade muito alta,SE típica do início do surto,"SE típica do início do surto - IC inferior (2,5%)","SE típica do início do surto - IC superior (97,5%)",duração típica do surto,"duração típica do surto - IC inferior (2,5%)","duração típica do surto - IC superior (97,5%)",temporadas utilizadas para os corredores endêmicos,ano,Unidade da Federação,Tipo,dado,escala
0,11,1805788,0.803458,0,0.103162,0.863224,1.168160,12.0,9.0,22.0,8.0,4.0,15.0,"2010, 2013, 2014, 2016",2017,Rondônia,Estado,srag,incidência
1,12,829619,0.953503,0,0.685939,1.783885,3.053363,24.0,5.0,49.0,6.0,3.0,16.0,"2010, 2011, 2012, 2013, 2014, 2015, 2016",2017,Acre,Estado,srag,incidência
2,13,4063614,0.119378,0,0.042171,0.165851,0.212193,12.0,4.0,25.0,12.0,6.0,13.0,"2010, 2012, 2013, 2014, 2015, 2016",2017,Amazonas,Estado,srag,incidência
3,14,522636,0.473615,0,0.414595,0.593964,0.801757,26.0,13.0,50.0,6.0,3.0,10.0,"2012, 2013, 2014, 2015, 2016",2017,Roraima,Estado,srag,incidência
4,15,8366628,0.256230,0,0.083865,0.466878,0.859727,12.0,10.0,18.0,10.0,5.0,13.0,"2012, 2014, 2015, 2016",2017,Pará,Estado,srag,incidência


In [34]:
dfs[dataset].keys()

Index(['UF', 'População',
       'Média geométrica do pico de infecção das temporadas regulares',
       'região de baixa atividade típica', 'limiar pré-epidêmico',
       'intensidade alta', 'intensidade muito alta',
       'SE típica do início do surto',
       'SE típica do início do surto - IC inferior (2,5%)',
       'SE típica do início do surto - IC superior (97,5%)',
       'duração típica do surto',
       'duração típica do surto - IC inferior (2,5%)',
       'duração típica do surto - IC superior (97,5%)',
       'temporadas utilizadas para os corredores endêmicos', 'ano',
       'Unidade da Federação', 'Tipo', 'dado', 'escala'],
      dtype='object')

In [35]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'Média geométrica do pico de infecção das temporadas regulares': 'geom_average_peak',
    'região de baixa atividade típica': 'low_activiy_region',
    'limiar pré-epidêmico': 'pre_epidemic_threshold',
    'intensidade alta': 'high_threshold',
    'intensidade muito alta': 'very_high_threshold',
    'SE típica do início do surto': 'epi_start',
    'SE típica do início do surto - IC inferior (2,5%)': 'epi_start_ci_lower',
    'SE típica do início do surto - IC superior (97,5%)': 'epi_start_ci_upper',
    'duração típica do surto': 'epi_duration',
    'duração típica do surto - IC inferior (2,5%)': 'epi_duration_ci_lower',
    'duração típica do surto - IC superior (97,5%)': 'epi_duration_ci_upper',
    'temporadas utilizadas para os corredores endêmicos': 'regular_seasons',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'Tipo': 'territory_type',  # Not needed in the table
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
regions_indeces = dfs[dataset].state_country.isin([
    'BR', 'RegN', 'RegL', 'RegC', 'RegS'
])
dfs[dataset].loc[regions_indeces, 'state_country'] = dfs[dataset].loc[
    regions_indeces, 'state_country'
].map(region_id)
dfs[dataset].state_country = dfs[dataset].state_country.astype(int)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)

dfs[dataset].head()

,state_country,population,geom_average_peak,low_activiy_region,pre_epidemic_threshold,high_threshold,very_high_threshold,epi_start,epi_start_ci_lower,epi_start_ci_upper,epi_duration,epi_duration_ci_lower,epi_duration_ci_upper,regular_seasons,year,dataset,scale
0,11,1805788,0.803458,0,0.103162,0.863224,1.168160,12.0,9.0,22.0,8.0,4.0,15.0,"2010, 2013, 2014, 2016",2017,1,1
1,12,829619,0.953503,0,0.685939,1.783885,3.053363,24.0,5.0,49.0,6.0,3.0,16.0,"2010, 2011, 2012, 2013, 2014, 2015, 2016",2017,1,1
2,13,4063614,0.119378,0,0.042171,0.165851,0.212193,12.0,4.0,25.0,12.0,6.0,13.0,"2010, 2012, 2013, 2014, 2015, 2016",2017,1,1
3,14,522636,0.473615,0,0.414595,0.593964,0.801757,26.0,13.0,50.0,6.0,3.0,10.0,"2012, 2013, 2014, 2015, 2016",2017,1,1
4,15,8366628,0.256230,0,0.083865,0.466878,0.859727,12.0,10.0,18.0,10.0,5.0,13.0,"2012, 2014, 2015, 2016",2017,1,1


In [36]:
dfs[dataset].tail()

,state_country,population,geom_average_peak,low_activiy_region,pre_epidemic_threshold,high_threshold,very_high_threshold,epi_start,epi_start_ci_lower,epi_start_ci_upper,epi_duration,epi_duration_ci_lower,epi_duration_ci_upper,regular_seasons,year,dataset,scale
187,0,207660929,26.289172,0,8.484101,49.118258,91.799762,16.0,5.0,21.0,12.0,11.0,15.0,"2010, 2012, 2013, 2014, 2015",2017,3,2
188,1003,27644815,4.280383,0,1.046539,4.452711,5.507278,15.0,7.0,20.0,10.0,5.0,18.0,"2010, 2012, 2013, 2014, 2015",2017,3,2
189,1002,67770757,2.771309,0,1.257426,3.756206,4.770510,16.0,8.0,30.0,8.0,3.0,17.0,"2010, 2011, 2012, 2013, 2014, 2015",2017,3,2
190,1001,16386007,4.228626,0,1.182767,6.055162,7.565655,12.0,2.0,16.0,8.0,1.0,11.0,"2010, 2012, 2013, 2014, 2015, 2016",2017,3,2
191,1004,95859350,6.935301,0,8.946847,11.831033,16.324451,25.0,17.0,33.0,10.0,7.0,14.0,"2010, 2011, 2014, 2015",2017,3,2


In [37]:
dfs[dataset].dtypes

state_country               int64
population                  int64
geom_average_peak         float64
low_activiy_region          int64
pre_epidemic_threshold    float64
high_threshold            float64
very_high_threshold       float64
epi_start                 float64
epi_start_ci_lower        float64
epi_start_ci_upper        float64
epi_duration              float64
epi_duration_ci_lower     float64
epi_duration_ci_upper     float64
regular_seasons            object
year                        int64
dataset                     int64
scale                       int64
dtype: object

In [38]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  geom_average_peak  \
dataset scale state_country year                                  
1       1     11            2017     1805788           0.803458   
              12            2017      829619           0.953503   
              13            2017     4063614           0.119378   
              14            2017      522636           0.473615   
              15            2017     8366628           0.256230   

                                  low_activiy_region  pre_epidemic_threshold  \
dataset scale state_country year                                               
1       1     11            2017                   0                0.103162   
              12            2017                   0                0.685939   
              13            2017                   0                0.042171   
              14            2017                   0                0.414595   
              15            2017                   0                0.083865   

                                  high_threshold  very_high_threshold  \
dataset scale state_country year                                        
1       1     11            2017        0.863224             1.168160   
              12            2017        1.783885             3.053363   
              13            2017        0.165851             0.212193   
              14            2017        0.593964             0.801757   
              15            2017        0.466878             0.859727   

                                  epi_start  epi_start_ci_lower  \
dataset scale state_country year                                  
1       1     11            2017       12.0                 9.0   
              12            2017       24.0                 5.0   
              13            2017       12.0                 4.0   
              14            2017       26.0                13.0   
              15            2017       12.0                10.0   

                                  epi_start_ci_upper  epi_duration  \
dataset scale state_country year                                     
1       1     11            2017                22.0           8.0   
              12            2017                49.0           6.0   
              13            2017                25.0          12.0   
              14            2017                50.0           6.0   
              15            2017                18.0          10.0   

                                  epi_duration_ci_lower  \
dataset scale state_country year                          
1       1     11            2017                    4.0   
              12            2017                    3.0   
              13            2017                    6.0   
              14            2017                    3.0   
              15            2017                    5.0   

                                  epi_duration_ci_upper  \
dataset scale state_country year                          
1       1     11            2017                   15.0   
              12            2017                   16.0   
              13            2017                   13.0   
              14            2017                   10.0   
              15            2017                   13.0   

                                                           regular_seasons  
dataset scale state_country year                                            
1       1     11            2017                    2010, 2013, 2014, 2016  
              12            2017  2010, 2011, 2012, 2013, 2014, 2015, 2016  
              13            2017        2010, 2012, 2013, 2014, 2015, 2016  
              14            2017              2012, 2013, 2014, 2015, 2016  
              15            2017                    2012, 2014, 2015, 2016

## 6. mem-typical

In [39]:
dataset = 'mem-typical'

In [40]:
dfs[dataset].head()

,UF,População,epiweek,corredor baixo,corredor mediano,corredor alto,ano,Unidade da Federação,Tipo,dado,escala
0,11,1805788,1,0.0,0.013703,0.041109,2017,Rondônia,Estado,srag,incidência
1,11,1805788,2,0.0,0.000000,0.000000,2017,Rondônia,Estado,srag,incidência
2,11,1805788,3,0.0,0.013703,0.041109,2017,Rondônia,Estado,srag,incidência
3,11,1805788,4,0.0,0.028790,0.087644,2017,Rondônia,Estado,srag,incidência
4,11,1805788,5,0.0,0.000000,0.000000,2017,Rondônia,Estado,srag,incidência


In [41]:
dfs[dataset].keys()

Index(['UF', 'População', 'epiweek', 'corredor baixo', 'corredor mediano',
       'corredor alto', 'ano', 'Unidade da Federação', 'Tipo', 'dado',
       'escala'],
      dtype='object')

In [42]:
migration_rules = {
    'UF': 'state_country',
    'População': 'population',
    'corredor baixo': 'low',
    'corredor mediano': 'median',
    'corredor alto': 'high',
    'ano': 'year',
    'Unidade da Federação': 'state_country_name',  # Not needed in the table
    'Tipo': 'territory_type',  # Not needed in the table
    'dado': 'dataset',
    'escala': 'scale'
}

dfs[dataset].rename(
    columns=migration_rules, inplace=True
)
# apply categories
dfs[dataset].dataset = dfs[dataset].dataset.map(dataset_id)
dfs[dataset].scale = dfs[dataset].scale.map(scale_id)
regions_indeces = dfs[dataset].state_country.isin([
    'BR', 'RegN', 'RegL', 'RegC', 'RegS'
])
dfs[dataset].loc[regions_indeces, 'state_country'] = dfs[dataset].loc[
    regions_indeces, 'state_country'
].map(region_id)
dfs[dataset].state_country = dfs[dataset].state_country.astype(int)

# remove unnecessary fields
dfs[dataset].drop([
    'state_country_name', 'territory_type'
], axis=1, inplace=True)

dfs[dataset].head()

,state_country,population,epiweek,low,median,high,year,dataset,scale
0,11,1805788,1,0.0,0.013703,0.041109,2017,1,1
1,11,1805788,2,0.0,0.000000,0.000000,2017,1,1
2,11,1805788,3,0.0,0.013703,0.041109,2017,1,1
3,11,1805788,4,0.0,0.028790,0.087644,2017,1,1
4,11,1805788,5,0.0,0.000000,0.000000,2017,1,1


In [43]:
dfs[dataset].tail()

,state_country,population,epiweek,low,median,high,year,dataset,scale
9979,1004,95859350,48,0.000000,0.506813,1.500926,2017,3,2
9980,1004,95859350,49,0.194681,1.032562,1.871175,2017,3,2
9981,1004,95859350,50,0.208623,1.026247,1.844569,2017,3,2
9982,1004,95859350,51,0.269861,0.779518,1.289445,2017,3,2
9983,1004,95859350,52,0.000000,0.000000,0.000000,2017,3,2


In [44]:
dfs[dataset].dtypes

state_country      int64
population         int64
epiweek            int64
low              float64
median           float64
high             float64
year               int64
dataset            int64
scale              int64
dtype: object

In [45]:
# primary_keys
pks[dataset] = ['dataset', 'scale', 'state_country', 'year', 'epiweek']

dfs[dataset].set_index(pks[dataset], inplace=True)
dfs[dataset].head()

population  low    median      high
dataset scale state_country year epiweek                                     
1       1     11            2017 1           1805788  0.0  0.013703  0.041109
                                 2           1805788  0.0  0.000000  0.000000
                                 3           1805788  0.0  0.013703  0.041109
                                 4           1805788  0.0  0.028790  0.087644
                                 5           1805788  0.0  0.000000  0.000000

## SQL Migration

In [46]:
exception_type_field = {
    'run_date': 'DATE'
}

In [47]:
def to_sql(df: pd.DataFrame, name: str, output_file):
    sql_remove = 'DROP TABLE IF EXISTS %(table_name)s;\n' 
    sql_struct = 'CREATE TABLE IF NOT EXISTS %(table_name)s (\n%(fields)s\n);\n' 
    sql_insert = 'INSERT INTO %(table_name)s (%(fields)s) VALUES (%(values)s);\n'
    
    sql_inserts = ''
    
    fields = ''
    header = ''
        
    dict_type = {
        'float64': 'FLOAT',
        'int64': 'INTEGER', 
        'object': 'VARCHAR'
    }
    
    num_type = ['DOUBLE', 'INTEGER']
    
    pks = list(df.index.names)
    
    df = df.reset_index()
    
    fields_name = ', '.join(df.keys())
    fields_type = {}
    
    name_max_len = len(max(df.keys(), key=lambda v: len(v)))
    
    for k in df.keys():
        # creating table
        if k in exception_type_field.keys():
            _type = exception_type_field[k]
        else:
            _type = dict_type[str(df[k].dtype)]
            
        fields_type[k] = _type
        
        field_settings = {
            'name': k.ljust(name_max_len, ' '),
            'type': _type.ljust(7, ' '),
            'null': 'NOT NULL' if (df[k].isnull() == False).all() else 'NULL'
        }
        
        if field_settings['type'] == 'varchar':
            field_settings['type'] += '(%s)' % int(df[k].str.len().max())
            
        fields += header + ('    %(name)s    %(type)s    %(null)s' % field_settings)
        
        header = ',\n'
    
    # inserting data
    for i, v_row in df.iterrows():
        values = []
        
        for k, v_col in v_row.items():
            values += [
                '%s' % v_col if fields_type[k] in num_type else 
                "'%s'" % v_col
            ]
        
        sql_inserts += sql_insert % {
            'table_name': name, 
            'fields': fields_name,
            'values': ','.join(values)
        }
        
    fields += header + ('    PRIMARY KEY (%s)' % ', '.join(pks))
    
    output_file.write(
        sql_remove % {'table_name': name}
    )
    
    output_file.write(
        sql_struct % {'table_name': name, 'fields': fields}
    )
    output_file.write(sql_inserts)

In [48]:
# creating dataset dataframe
df_dataset = pd.DataFrame({
    'id': list(dataset_id.values()), 
    'name': list(dataset_id.keys())
}).set_index('id')

dfs['dataset'] = df_dataset

df_dataset

,name
id,
1,srag
2,sragflu
3,obitoflu


In [49]:
# creating situation dataframe
df_situation = pd.DataFrame({
    'id': list(situation_id.values()), 
    'name': list(situation_id.keys())
}).set_index('id')

dfs['situation'] = df_situation

df_situation

,name
id,
1,unknown
2,estimated
3,stable


In [50]:
# creating scale dataframe
df_scale = pd.DataFrame({
    'id': list(scale_id.values()), 
    'name': list(scale_id.keys())
}).set_index('id')

dfs['scale'] = df_scale

df_scale

,name
id,
1,incidência
2,casos


In [51]:
# creating territory_type dataframe
df_territory_type = pd.DataFrame({
    'id': list(territory_type_id.values()), 
    'name': list(territory_type_id.keys())
}).set_index('id')

dfs['territory_type'] = df_territory_type

df_territory_type

,name
id,
1,Estado
2,Regional
3,Região
4,País


In [52]:
dfs['territory'] = df_territory

df_territory.head()

,initials,name,territory_type
id,,,
11,RO,Rondônia,1
12,AC,Acre,1
13,AM,Amazonas,1
14,RR,Roraima,1
15,RO,Pará,1


In [ ]:
dsn = 'postgresql://%(USER)s:%(PASSWORD)s@%(HOST)s/%(NAME)s'
engine = sqla.create_engine(dsn % settings.DATABASE)
    
for k, df in dfs.items():
    k_new = k.replace('-', '_')
    
    print('Migrating %s ...' % k_new)
    df.to_sql(
        k_new, engine, if_exists='replace',
        chunksize=2048
    )
        

Migrating mem_report ...
Migrating current_estimated_values ...
Migrating clean_data_epiweek_weekly_incidence_w_situation ...
